In [1]:
#Load Module
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import time
import copy


In [2]:
# Get files name
path1 = 'train1/'
lenght = 47

sg_files = []
spike_files = []

for i in range(1, lenght):
    sg_files.append( path1 + str(i) + '_sg.csv')
    spike_files.append( path1 + str(i) + '_spike.csv')
    
print(sg_files)
print(spike_files)


['train1/1_sg.csv', 'train1/2_sg.csv', 'train1/3_sg.csv', 'train1/4_sg.csv', 'train1/5_sg.csv', 'train1/6_sg.csv', 'train1/7_sg.csv', 'train1/8_sg.csv', 'train1/9_sg.csv', 'train1/10_sg.csv', 'train1/11_sg.csv', 'train1/12_sg.csv', 'train1/13_sg.csv', 'train1/14_sg.csv', 'train1/15_sg.csv', 'train1/16_sg.csv', 'train1/17_sg.csv', 'train1/18_sg.csv', 'train1/19_sg.csv', 'train1/20_sg.csv', 'train1/21_sg.csv', 'train1/22_sg.csv', 'train1/23_sg.csv', 'train1/24_sg.csv', 'train1/25_sg.csv', 'train1/26_sg.csv', 'train1/27_sg.csv', 'train1/28_sg.csv', 'train1/29_sg.csv', 'train1/30_sg.csv', 'train1/31_sg.csv', 'train1/32_sg.csv', 'train1/33_sg.csv', 'train1/34_sg.csv', 'train1/35_sg.csv', 'train1/36_sg.csv', 'train1/37_sg.csv', 'train1/38_sg.csv', 'train1/39_sg.csv', 'train1/40_sg.csv', 'train1/41_sg.csv', 'train1/42_sg.csv', 'train1/43_sg.csv', 'train1/44_sg.csv', 'train1/45_sg.csv', 'train1/46_sg.csv']
['train1/1_spike.csv', 'train1/2_spike.csv', 'train1/3_spike.csv', 'train1/4_spike.csv',

In [3]:
# Read file and seperate each feature to do data preprocessing
s = time.time()
print('Concating sg...')
sg_data = [pd.read_csv(i) for i in sg_files]

print('Concating spike')
spike_data = [pd.read_csv(i) for i in spike_files]

print('Reading MaxWear')
output = pd.read_csv(path1+'00_Wear_data.csv')
output.drop(['Index'], axis="columns", inplace=True)

print('sg_data: ' + str(len(sg_data)))
print('spike_data: ' + str(len(spike_data)))

print('cost time:', time.time() - s)


Concating sg...
Concating spike
Reading MaxWear
sg_data: 46
spike_data: 46
cost time: 49.41358232498169


In [4]:
# Define feature index
Sg_Feature_Index = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I']
Spike_Feature_Index = ['A', 'B', 'C', 'D']

s = time.time()

# Concat data
sg_data_concat = pd.concat(sg_data, axis=1)
spike_data_concat = pd.concat(spike_data, axis=1)

print('sg_data_concat' + str(sg_data_concat.shape))
print('spike_data_concat' + str(spike_data_concat.shape))

# Create dict then use the last value of that column to fill in to get a same dimention in one featrue(column)
sg = {}
spike = {}

for i in Sg_Feature_Index:
  data = sg_data_concat[i]
  for j in range(0, lenght-1):
      initial_data = data.iloc[:, j]
      if True in initial_data.isnull().values:
          last_value = initial_data[np.where(
              initial_data.isnull().values == True)[0][0]-1]
          new_data = data.iloc[:, j].fillna(last_value).copy()
          data.iloc[:, j] = new_data
  sg['sg_' + str(i)] = data
  

for i in Spike_Feature_Index:
  data = spike_data_concat[i]
  for j in range(0, lenght-1):
      initial_data = data.iloc[:, j]
      if True in initial_data.isnull().values:
          last_value = initial_data[np.where(
              initial_data.isnull().values == True)[0][0]-1]
          new_data = data.iloc[:, j].fillna(last_value).copy()
          data.iloc[:, j] = new_data
  spike['spike_' + str(i)] = data

print('cost time:', time.time() - s)


sg_data_concat(416001, 414)
spike_data_concat(1040001, 184)


C:\Users\jasmi\AppData\Local\Temp\ipykernel_12492\1743705112.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = new_data
C:\Users\jasmi\AppData\Local\Temp\ipykernel_12492\1743705112.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = new_data
C:\Users\jasmi\AppData\Local\Temp\ipykernel_12492\1743705112.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = new_data
C:\Users\jasmi\A

In [53]:
print(sg.keys())
print(spike.keys())

# np.save()
sg_feture = list(sg.keys())
spike_feture = list(spike.keys())


# spike['spike_A']
# for i in spike_feture:
#     print(i)
#     data = spike[i]
#     for j in range(0, data.shape[1]-1, 5):
#         print(data.iloc[:, j].shape)
#         value = [0]
#         for k in range(3, data.iloc[:, j].shape[0]):
#             print(data.iloc[:, j][k] + data.iloc[:, j][k-1]/2)
#         spike[i].iloc[:, j] = value

# for i in sg_feture:
#     np.save(i, sg[i])

# for i in spike_feture:
#     np.save(i, spike[i])


dict_keys(['sg_A', 'sg_B', 'sg_C', 'sg_D', 'sg_E', 'sg_F', 'sg_H', 'sg_I'])
dict_keys(['spike_A', 'spike_B', 'spike_C', 'spike_D'])


TypeError: insert() missing 1 required positional argument: 'value'

In [102]:

data = spike['spike_A'].iloc[:, 0]
new_spike = [0]
index = range(3, data.shape[0]-1, 5)
data = list(data)
value = [(data[j-1] + data[j])/2 for j in index]
data = np.insert(data, index, value)
new_data = [data[i] for i in range(0, len(data), 3)]
new_data = pd.DataFrame(new_data, columns=['A'])

new_data


,A
0,0.000
1,0.001
2,0.002
3,0.003
4,0.004
...,...
415996,415.996
415997,415.997
415998,415.998
415999,415.999
